# Integrate Griptape with Astra DB Serverless

For more information, see the DataStax [Astra DB docs page](https://docs.datastax.com/en/astra-db-serverless/integrations/griptape.html).

In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools

!pip install "griptape[drivers-vector-astra-db,drivers-web-scraper-trafilatura]" "python-dotenv==1.0.1"

## Prerequisites

- Have the API Endpoint for your Astra DB
- Have an Application Token for your Astra DB
- Create a collection called `"griptape_integration"` with "Bring your own" embeddings and dimension 1536
- Have an OpenAI API Key

## Secrets

Example values:

- API Endpoint: `"https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com"`
- Token: `"AstraCS:6gBhNmsk135..."`
- OpenAI API key: `"sk-..."`

In [ ]:
import os
from getpass import getpass


os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")
os.environ["ASTRA_DB_API_ENDPOINT"] = input("ASTRA_DB_API_ENDPOINT = ")
if _desired_namespace := input("ASTRA_DB_NAMESPACE (optional) = "):
    os.environ["ASTRA_DB_NAMESPACE"] = _desired_namespace

default_collection_name = "griptape_integration"
os.environ["GRIPTAPE_COLLECTION_NAME"] = input("GRIPTAPE_COLLECTION_NAME (empty for default) = ") or default_collection_name
os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY = ")

## Dependencies

In [ ]:
from griptape.drivers import (
    AstraDbVectorStoreDriver,
    OpenAiChatPromptDriver,
    OpenAiEmbeddingDriver,
)
from griptape.engines.rag import RagEngine
from griptape.engines.rag.modules import (
    PromptResponseRagModule,
    VectorStoreRetrievalRagModule,
)
from griptape.engines.rag.stages import ResponseRagStage, RetrievalRagStage
from griptape.loaders import WebLoader
from griptape.structures import Agent
from griptape.tools import RagClient, TaskMemoryClient

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]
ASTRA_DB_API_ENDPOINT = os.environ["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_NAMESPACE = os.environ.get("ASTRA_DB_NAMESPACE")
GRIPTAPE_COLLECTION_NAME = os.environ["GRIPTAPE_COLLECTION_NAME"]

## Initialize Vector Store and RAG Engine

In [ ]:
namespace = "datastax_blog"

vector_store_driver = AstraDbVectorStoreDriver(
    embedding_driver=OpenAiEmbeddingDriver(),
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    collection_name=GRIPTAPE_COLLECTION_NAME,
    astra_db_namespace=ASTRA_DB_NAMESPACE,
)

engine = RagEngine(
    retrieval_stage=RetrievalRagStage(
        retrieval_modules=[
            VectorStoreRetrievalRagModule(
                vector_store_driver=vector_store_driver,
                query_params={
                    "count": 2,
                    "namespace": namespace,
                },
            )
        ]
    ),
    response_stage=ResponseRagStage(
        response_module=PromptResponseRagModule(
            prompt_driver=OpenAiChatPromptDriver(model="gpt-4o")
        )
    )
)

## Ingest a Web page in the Vector Store

In [ ]:
input_blogpost = (
    "www.datastax.com/blog/indexing-all-of-wikipedia-on-a-laptop"
)

vector_store_driver.upsert_text_artifacts(
    {namespace: WebLoader(max_tokens=256).load(input_blogpost)}
)

## Pass the RAG Engine as a tool to an Agent

In [ ]:
vector_store_tool = RagClient(
    description="A DataStax blog post",
    rag_engine=engine,
)
agent = Agent(tools=[vector_store_tool, TaskMemoryClient(off_prompt=False)])

## Have the Agent run a RAG query on the Vector Store

In [ ]:
agent.run(
    "What engine made possible to index such an amount of data, "
    "and what kind of tuning was required?"
)

answer = agent.output_task.output.value

print(answer)